In [2]:
import numpy as np


F = GF(9, 'alpha')
alpha = F.gen() 
d = F.order() // 2 + 1


def check_inequality(n, r):
    return r == n


def make_weight1(n, F):
    slots = list(range(n))
    P = matrix(F, n, n, 0)
    for i in range(n):
        index = choice(slots)
        slots.remove(index)
        P[i, index] = 1
    return P


def Schur_product(M, k1):
    Schur = []
    for i in range(k1):
        a_row = np.array(list(M[i]))
        for j in range(i, k1):
            b_row = np.array(list(M[j]))
            Schur.append(list(np.multiply(a_row, b_row)))
    return Schur


def generate_random_non_singular_matrix(F, k1):
    S = random_matrix(F, k1, k1)
    while S.is_singular():
        S = random_matrix(F, k1, k1)
    return S


def SchurCalc(M, F):
    k1 = M.rank()
    n = M.ncols()
    S = generate_random_non_singular_matrix(F, k1)
    P1 = make_weight1(n, F)
    G_cap_hidden1 = S * M * P1
    G_cap_Schur1 = Matrix(F, Schur_product(G_cap_hidden1, k1))
    G1k = G_cap_Schur1.rank()
    return check_inequality(n, G1k)


def codegen(F, d, a, b, c):
    res = [None] * 5
    A = F(a)
    B = F(b) + F(c)**2 / F(4)
    disc = -16 * (4 * A**3 + 27 * B**2)
    if disc == 0:
        raise ValueError("Сингулярная кривая: дискриминант равен 0")
    # кривая
    R.<x, y> = PolynomialRing(F, 2)
    f = y**2 + c * y - (x**3 + a * x + b)
    C = Curve(f)
    # функциональное поле и места порядка 1
    F_field = C.function_field()
    places = F_field.places(1)
    if len(places) <= 1:
        raise ValueError("Недостаточно мест для построения AG-кода")
    # дивизор 
    Q = places[0]
    del places[0]
    G = d * Q
    # AG-код
    code = codes.EvaluationAGCode(places, G)
    res[0] = SchurCalc(code.generator_matrix(), F)
    # Проколотый код 
    try:
        code1 = codes.PuncturedCode(code, 1)
        res[1] = SchurCalc(code1.generator_matrix(), F)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить проколотый")
    # Укороченный код
    try:
        code2 = code1.shortened([1])
        res[2] = SchurCalc(code2.generator_matrix(), F)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить укороченный")
    # Подполевой код
    code3 = codes.SubfieldSubcode(code, F)
    res[3] = SchurCalc(code3.generator_matrix(), F)
    # Трэйс-код
    code41 = code.dual_code()
    code42 = codes.SubfieldSubcode(code41, F)
    code43 = code42.dual_code()
    res[4] = SchurCalc(code43.generator_matrix(), F)
    return res


# -------------------- ОСНОВНОЙ БЛОК --------------------
number_tests = 0   # количество успешных тестов
probs = [0] * 5    # средние значения (доли успеха) для пяти типов кодов
total_tests = 5  # количество попыток

while number_tests < total_tests:
    a = F.random_element()
    b = F.random_element()
    c = F.random_element()
    try:
        res = codegen(F, d, a, b, c)
        if None not in res:
            for i in range(5):
                probs[i] = (probs[i] * number_tests + (1 if res[i] else 0)) / (number_tests + 1)
            number_tests += 1
    except Exception as e:
        print("error is", e)
        continue

code_names = ['AG-code', 'Punctured-code', 'Shortened-code', 'Subfield-code', 'Trace-code']
for i in range(5):
    print(f"{code_names[i]}: {probs[i]}")

error is Сингулярная кривая: дискриминант равен 0
AG-code: 3/5
Punctured-code: 3/5
Shortened-code: 3/5
Subfield-code: 3/5
Trace-code: 3/5
